In [1]:
from google.colab import drive
drive.mount('/content/d')

Mounted at /content/d


Задача собрать информацию (в этом примере - координаты) по нескольким физическим адресам через API *Google Maps*. Возьмем адреса нескольких ведущих москвоских ВУЗов, соберем координаты, визуализируем на карте.

In [2]:
adrDict = {'MSU': '119991, Российская Федерация, Москва, Ленинские горы, д. 1',
           'MIPT': '41701, Московская область, г. Долгопрудный, Институтский переулок, д.9.',
           'HSE': 'Россия, 101000, г. Москва, ул. Мясницкая, д. 20'}

##### **ШАГ 1**: заберем свой API ключ Google Maps.

Сделать это можно на сайте **Google Cloud Platform**. Для этого:

1. Идем на сайт [Google Cloud Credentials](https://console.cloud.google.com/project/_/apiui/credential)

2. Переходим *+ CREATE CREDENTIALS* -> *API key*

3. Далее необходимо авторизовать API для сервиса Geocoding. для этого переходим на [Google Maps Platform](https://console.cloud.google.com/google/maps-apis/overview) -> *APIs* , ищем Geocoding and кликаем *Google Maps Geocoding API* -> *Enable API*

После этого можно сохранить ключ у себяи загрузить его в переменную.

In [3]:
with open('d/My Drive/googleMap/apikey.txt') as f:
    apiKey = f.readline()
    f.close

##### **ШАГ 2**: отправить запрос и получить ответ через  **REQUEST**

In [4]:
import requests # простая HTTP библиотека для запросов
from collections import defaultdict # удобная фукнция для хранения массивов даже без заранее прописанного ключа

In [5]:
def getInfo(adrDict):
    '''
    забираем и сохраняем в json координаты с google maps
    '''
    for key in adrDict.keys(): # для каждого указанного места
        query = adrDict[key] # берем физический адрес
        # и вставляем в запрос
        url = 'https://maps.googleapis.com/maps/api/geocode/json?address=' + query + '&lang=ru&key=' + apiKey
        data[key] = requests.get(url).json() # сохраняем результат как json

    return data

In [6]:
data = defaultdict(dict) # обработанные координаты кидаем в один словарь
getInfo(adrDict)

defaultdict(dict,
            {'HSE': {'results': [{'address_components': [{'long_name': '20',
                  'short_name': '20',
                  'types': ['street_number']},
                 {'long_name': 'Myasnitskaya Ulitsa',
                  'short_name': 'Myasnitskaya Ulitsa',
                  'types': ['route']},
                 {'long_name': 'Tsentralnyy administrativnyy okrug',
                  'short_name': 'Tsentralnyy administrativnyy okrug',
                  'types': ['political',
                   'sublocality',
                   'sublocality_level_1']},
                 {'long_name': 'Moskva',
                  'short_name': 'Moskva',
                  'types': ['locality', 'political']},
                 {'long_name': 'Basmannyy',
                  'short_name': 'Basmannyy',
                  'types': ['administrative_area_level_3', 'political']},
                 {'long_name': 'Moskva',
                  'short_name': 'Moskva',
                  'types': ['a

Теперь визуализируем это на карте

##### **ШАГ 3**: визиализируем с помочщью *Folium*


Для удобства долготу и ширину географических объектов размещаю в датафрейме

In [7]:
import pandas as pd

adrDf = pd.DataFrame(
    columns = ['Address', 'Latitude', 'Longtitude'],
    index = list(data.keys())
)

for key in data.keys():
    adrDf.loc[key]['Address'] = adrDict[key] 
    adrDf.loc[key]['Latitude'] = data[key]['results'][0]['geometry']['location']['lat']
    adrDf.loc[key]['Longtitude'] = data[key]['results'][0]['geometry']['location']['lng']

print(adrDf) 

                                                Address Latitude Longtitude
MSU   119991, Российская Федерация, Москва, Ленински...  55.7029    37.5304
MIPT  41701, Московская область, г. Долгопрудный, Ин...  55.9296    37.5158
HSE     Россия, 101000, г. Москва, ул. Мясницкая, д. 20  55.7616    37.6333


[Folium](http://python-visualization.github.io/folium/) очень удобная и простая для начинающих библиотека, с помощью которой можно визуализировать адреса на *интерактивной карте*. Нет нужды подгружать дополнительные исходники, просто вызываем функцию *folium.Map*. Передаем координаты, ставим исходный масштаб приближения и готово:

In [8]:
import folium
from folium.plugins import MarkerCluster # функция анимированной кластеризации

# инициализируем карту
m = folium.Map(
    # берем среднее по кооридантам для центровки
    location = adrDf[['Latitude', 'Longtitude']].mean().to_list(), #
    zoom_start=10 # ставим исходное приближение
)

markerCluster = MarkerCluster().add_to(m) # инициализируем кластеризацию

# собираем данные из датафрейма
for idx in adrDf.index:
    location = (adrDf.loc[idx]['Latitude'], adrDf.loc[idx]['Longtitude']) # наши координаты
    folium.Marker(
        location = location,
        popup = adrDf.loc[idx]['Address'], # имя точки
        tooltip = adrDf.loc[idx]['Address'], # текст поверх точек
    ).add_to(markerCluster)

m # рисуем карту

Сохраняем в HTML:

In [9]:
m.save('2021-05-23-google-maps-api-simple-address-querry.html')

In [10]:
from google.colab import files
files.download('2021-05-23-google-maps-api-simple-address-querry.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>